In [87]:
import pandas as pd
import math
from scipy import spatial
import libpysal as ps
import numpy as np
from pointpats import PointPattern
from pointpats import PoissonPointProcess, PoissonClusterPointProcess, Window, poly_from_bbox, PointPattern
import libpysal as ps
from libpysal.cg import shapely_ext
%matplotlib inline
import matplotlib.pyplot as plt
import pointpats.quadrat_statistics as qs
from pointpats import PointPattern, as_window
from pointpats import PoissonPointProcess as csr
from pointpats.geometry import *
from pointpats.geometry import (prepare_hull as _prepare_hull,     area as _area,
    k_neighbors as _k_neighbors,
    build_best_tree as _build_best_tree,
    prepare_hull as _prepare_hull,
    TREE_TYPES,
)
from pointpats.ripley import _prepare # very important to prepare data :)

In [65]:
def kinhom(coords_array):
    coordinates, support, distances, metric, hull, edge_correction = _prepare(
            coords_array, support=None,
        distances=None,
        metric="euclidean",
        hull=None,
        edge_correction=None,
        )

    hull = _prepare_hull(coords_array, hull = None)

    n = coordinates.shape[0]
    upper_tri_distances = spatial.distance.pdist(coordinates, metric='euclidean') # OK  DISTANCES 
    n_pairs_less_than_d = (upper_tri_distances < support.reshape(-1, 1)).sum(axis=1) # N PAIRS LESS THAN DISTANCES 
    lambda_1 = n / _area(hull) #INTENSITY 
    k_estimate = ((n_pairs_less_than_d * 2) / n) / lambda_1 # OK  ((N PAIRS LESS THAN DISTANCE * 2) / N POINTS )/ INTENSITY    #BRING UP THE POINT FROM BADDELEY THAT ITS SHOULD BE N-1\HULL AREAS (PP 204)
    uno_fratto_a = 1/ _area(hull)
    lambda_2 = (n * (n - 1)) / (_area(hull))**2
    k_inhom_estimate = ((n_pairs_less_than_d *2) / (lambda_2)) * uno_fratto_a
    return support, k_inhom_estimate

# Calculate Kinhom on simulated points 

In [226]:
simulated_companies = pd.read_pickle('./simulated_companies.pkl')
simulated_companies.columns = [['lat', 'long']]
def coords_to_array(df): 
    df['array'] = df.apply(lambda x: np.array(list(zip(x['lat'], x['long']))), axis = 1) 
    return df
array_df = coords_to_array(simulated_companies)
list_of_arrays = array_df['array'].values.tolist()
kinhom_values = []
for x in list_of_arrays:
    for array in x: 
        kinhom_lista = kinhom(array)
        kinhom_values.append(kinhom_lista)
kinhom_estimates = pd.DataFrame(kinhom_values, columns = ['support', 'kinhom_estimate'])

# Calculate L transformation

In [96]:
def L_function(list_of_estimates):
    L = [math.sqrt(x/math.pi) for x in list_of_estimates]
    return L
kinhom_estimates['l_estimates'] = kinhom_estimates['kinhom_estimate'].apply(L_function)

In [99]:
kinhom_estimates

,support,kinhom_estimate,l_estimates
0,"[0.0, 0.005843940808605459, 0.0116878816172109...","[0.0, 0.0001255911684666141, 0.000753547010799...","[0.0, 0.006322729674776331, 0.0154874614847554..."
1,"[0.0, 0.004690301043119766, 0.0093806020862395...","[0.0, 0.0001289552533311945, 0.000386865759993...","[0.0, 0.006406850397087044, 0.0110969904042475..."
2,"[0.0, 0.004019622042128335, 0.0080392440842566...","[0.0, 0.0001336463525582218, 0.000133646352558...","[0.0, 0.006522342774777047, 0.0065223427747770..."
3,"[0.0, 0.008317952651485832, 0.0166359053029716...","[0.0, 0.00039152049783559454, 0.00091354782828...","[0.0, 0.011163549843336984, 0.0170526040605013..."
4,"[0.0, 0.012343536506338812, 0.0246870730126776...","[0.0, 0.00045996181682514176, 0.00383301514020...","[0.0, 0.012100016262902313, 0.0349297382329275..."
...,...,...,...
995,"[0.0, 0.00748284661663554, 0.01496569323327108...","[0.0, 0.00029271944182663353, 0.00146359720913...","[0.0, 0.009652745319939735, 0.0215841947048782..."
996,"[0.0, 0.010390779877309068, 0.0207815597546181...","[0.0, 0.00036711996074315203, 0.00357941961724...","[0.0, 0.010810083853511518, 0.0337544760138491..."
997,"[0.0, 0.004667899100341413, 0.0093357982006828...","[0.0, 0.00013795580038456656, 0.00048284530134...","[0.0, 0.006626665460003631, 0.0123973558840511..."
998,"[0.0, 0.0077147498361330785, 0.015429499672266...","[0.0, 0.0002095156374600483, 0.001466609462220...","[0.0, 0.00816644957822139, 0.02160639467832211..."


# Find Null L(d)s

## case data (digital) + distance calculation

In [232]:
list_of_arrays = simulated_companies['array'].values.tolist()
sampled_cases = [x[0][np.random.choice(x[0].shape[0], size=5, replace = False), :] for x in list_of_arrays]
simulated_companies['sample_cases'] = sampled_cases

In [201]:
x = np.array([[1,2],[3,4],[5,6]])
df = pd.DataFrame()
df["array"] = [x, x]
df['sampled_cases'] = df['array'].apply(lambda x : x[np.random.choice(x.shape[0], 2)])
df

,array,sampled_cases
0,"[[1, 2], [3, 4], [5, 6]]","[[3, 4], [1, 2]]"
1,"[[1, 2], [3, 4], [5, 6]]","[[1, 2], [1, 2]]"


## control data (all)

In [235]:
list_of_arrays = simulated_companies['array'].values.tolist()
sampled_controls = [x[0][np.random.choice(x[0].shape[0], size=5, replace = False), :] for x in list_of_arrays]
simulated_companies['sample_controls'] = sampled_controls

In [237]:
simulated_companies

,lat,long,array,sample_cases,sample_controls
0,"[51.54218582497672, 51.589472314705674, 51.381...","[-0.0874064825256965, -0.24605053217241302, -0...","[[51.54218582497672, -0.0874064825256965], [51...","[[51.540058860578284, -0.15367437950814467], [...","[[51.62686312200529, -0.20409681102874566], [5..."
1,"[51.55988941008741, 51.46151770699342, 51.4621...","[-0.2039481156403623, -0.10841744337054215, -0...","[[51.55988941008741, -0.2039481156403623], [51...","[[51.35327031169856, -0.24312197030527327], [5...","[[51.64812764556005, 0.0421331591972921], [51...."
2,"[51.56214130611294, 51.51280788687047, 51.3606...","[-0.12966170920812858, -0.2826523378177931, -0...","[[51.56214130611294, -0.12966170920812858], [5...","[[51.59572766560016, 0.012211467041787327], [5...","[[51.36063622075598, -0.1410989364580325], [51..."
3,"[51.56170641195442, 51.52023769828351, 51.4886...","[-0.166205136211394, -0.13644063967513753, -0....","[[51.56170641195442, -0.166205136211394], [51....","[[51.54525145853793, 0.07155797156247579], [51...","[[51.59649003453747, -0.113847583564047], [51...."
4,"[51.4906797623787, 51.497014131477584, 51.5169...","[-0.1896979435260494, -0.17099360746609682, -0...","[[51.4906797623787, -0.1896979435260494], [51....","[[51.47399498096957, -0.07408149733248026], [5...","[[51.53715648758911, -0.13194912267099246], [5..."
...,...,...,...,...,...
995,"[51.63935421745579, 51.706750976828324, 51.542...","[-0.07261284013816416, -0.009181363548142474, ...","[[51.63935421745579, -0.07261284013816416], [5...","[[51.436794597752765, 0.044096045478457024], [...","[[51.52995463041917, -0.09372235010363428], [5..."
996,"[51.61569290725654, 51.63541106925196, 51.5147...","[0.11716453816526606, 0.122925519204793, -0.16...","[[51.61569290725654, 0.11716453816526606], [51...","[[51.50069525264207, -0.19122464053460198], [5...","[[51.45793383096564, -0.21708944092804938], [5..."
997,"[51.52412280586476, 51.491516413109416, 51.459...","[-0.14427792343017554, -0.3858098677670566, 0....","[[51.52412280586476, -0.14427792343017554], [5...","[[51.56718379921012, -0.2482630947199408], [51...","[[51.616346064199846, -0.006877216491134486], ..."
998,"[51.57654497729315, 51.54095405036929, 51.5460...","[0.05455564471325014, -0.27824276403446013, -0...","[[51.57654497729315, 0.05455564471325014], [51...","[[51.63956379417189, -0.06530279121795612], [5...","[[51.63956379417189, -0.06530279121795612], [5..."


In [250]:
# calc distances(d) on sample cases 

In [247]:
def distances(coords_array):
    coordinates, support, distances, metric, hull, edge_correction = _prepare(
            coords_array, support=None,
        distances=None,
        metric="euclidean",
        hull=None,
        edge_correction=None,
        )

    upper_tri_distances = spatial.distance.pdist(coordinates, metric='euclidean') # OK  DISTANCES 
    n_pairs_less_than_d = (upper_tri_distances < support.reshape(-1, 1)).sum(axis=1) # N PAIRS LESS THAN DISTANCES 
    return n_pairs_less_than_d

In [255]:
list_of_arrays = simulated_companies['sample_cases'].values.tolist()
distance = []
for x in list_of_arrays:
    for array in x: 
        distances0 = distances(array)
        distance.append(distances0)

# cal lambda(X) on sample controls 
STILL NOT WORKING

In [252]:
def lambdas(coords_array, distance):
    coordinates, support, distances, metric, hull, edge_correction = _prepare(
            coords_array, support=None,
        distances=None,
        metric="euclidean",
        hull=None,
        edge_correction=None,
        )

    hull = _prepare_hull(coords_array, hull = None)

    n = coordinates.shape[0]
    uno_fratto_a = 1/ _area(hull)
    lambda_2 = (n * (n - 1)) / (_area(hull))**2
    k_inhom_estimate = ((distance *2) / (lambda_2)) * uno_fratto_a
    return k_inhom_estimate

In [257]:
list_of_controls = simulated_companies['sample_controls'].values.tolist()
kinhom_ests = []
for x in list_of_controls:
    for array in x: 
        lambdas = lambdas(array, distance)
        kinhom_ests.append(lambdas)

TypeError: 'numpy.ndarray' object is not callable